## Qiskit Gradient Framework

The gradient framework enables the evaluation of quantum gradients as well as functions thereof.
Besides standard first order gradients of expectation values of the form
$$ \langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle $$
<!--- $$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} $$

$$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta^2}, $$
--->

the gradient also supports the evaluation of second order gradients (Hessians), and the Quantum Fisher Information (QFI) of pure quantum states $|\psi\left(\theta\right)\rangle$.

### First Order Gradients

Three types of first order gradients are supported by the gradient framework.
1. Gradient of an expectation value w.r.t. a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega} $
2.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} $
3.  Gradient of sampling probabilities w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial p_i}{\partial\theta} = \frac{\partial\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta} $

#### Gradients w.r.t. Measurement Operator Coefficients

Given a parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$, and observable $\hat{O}\left(\omega\right)=\sum_{i}\omega_i\hat{O}_i$, we want to compute 
$$ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega_i} = \langle\psi\left(\theta\right)|\hat{O}_i\left(\omega\right)|\psi\left(\theta\right)\rangle. $$



In [1]:
import numpy as np
from qiskit.aqua.operators import Z, X, I, StateFn, CircuitStateFn, SummedOp
from qiskit.aqua.operators.gradients import Gradient, NaturalGradient, QFI, Hessian
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression
from qiskit.circuit.library import EfficientSU2

In [2]:
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

In [3]:
# Instantiate the Hamiltonian observable
coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = coeff_0 * X + coeff_1 * Z

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

# Define the coefficients w.r.t. we want to compute the gradient
gradient_coeffs = [coeff_0, coeff_1]

# Convert the operator and the gradient target coefficients into the respective operator
grad = Gradient().convert(operator = op, params = gradient_coeffs)

# Define the values to be assigned to the parameters
value_dict = {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi}

# Assign the parameters and evaluate the gradient
grad_result = grad.assign_parameters(value_dict).eval()
print('Gradient ', grad_result)

Gradient  [(0.7071067811865475+5.6e-17j), 0j]


#### Gradients w.r.t. Measurement Quantum State Parameters

Next we consider gradients w.r.t. $\theta$, i.e., 
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}, $
 
 respectively,
 
  $ \frac{\partial p_i}{\partial\theta} = \frac{\partial\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta}. $

Notably, the latter case does not use $2^n$ projective operators but is based on sampled values from adaptions of $|\psi\left(\theta\right)\rangle$. 

There are several ways to compute this type of gradient.
* Parameter shifts: This method does not require additional working qubits but the evaluation of multiple expectation values.
* Finite difference: This method compute a numerical approximation rather than an analytic gradient. Equivalently, to the parameter shifting this method does not require additional working qubits but the evaluation of multiple expectation values.
* Linear combination of unitaries: This method requires a single circuit to compute the gradient but uses one additional working qubit and intercepting controlled quantum gates.

##### Parameter Shift Gradients
<a id='param_shift_grad'></a>
Given a Hermitian operator $g$ with two unique eigenvalues $\pm r$ which acts as generator for a parameterized quantum gate $$G(\theta)= e^{-i\theta g}.$$
Then, quantum gradients can be computed by using eigenvalue $r$ dependent shifts to parameters. All [standard, parameterized qiskit gates](https://github.com/Qiskit/qiskit-terra/tree/master/qiskit/circuit/library/standard_gates) can be shifted with $\pi/2$, i.e.,
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} = 2 \left(\langle\psi\left(\theta+\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\pi/2\right)\rangle - \right.$
 $\left.\partial\langle\psi\left(\theta-\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\pi/2\right)\rangle\right).$
 Probability gradients are computed equivalently.

In [4]:
# Define the Hamiltonian with fixed coefficients
H = 0.5 * X - 1 * Z
# Define the parameters w.r.t. we want to compute the gradients
params = [a, b]

In [5]:
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
state_grad_op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
state_grad = Gradient(method='param_shift').convert(operator=state_grad_op, params=params)
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with parameter shift', state_grad_result)

State gradient computed with parameter shift [(-0.35355339059327373-4.175e-17j), (0.7071067811865475+4.15e-17j)]


##### Finite Difference Gradients

<a id='fin_diff_grad'></a>

Unlike the other methods, finite difference gradients are numerical estimations rather than analytical values.
This implementation employs a central difference approach with $\epsilon << 1$
 $ \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta} \approx \frac{1}{2\epsilon} \left(\langle\psi\left(\theta+\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta+\epsilon\right)\rangle - \partial\langle\psi\left(\theta-\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta-\epsilon\right)\rangle\right).$
 Probability gradients are computed equivalently.

In [6]:
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
state_grad_op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
state_grad = Gradient(method='fin_diff').convert(operator=state_grad_op, params=params)
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with finite difference', state_grad_result)

State gradient computed with finite difference [(-0.3535533905669581-2.775e-11j), (0.7071067812098961+2.7749999999999998e-11j)]


##### Linear Combination of Unitaries Gradients
<a id='lin_comb_grad'></a>
Unitaries can be written as $U\left(\omega\right) = e^{iM\left(\omega\right)}$, where $M\left(\omega\right)$ denotes a parameterized Hermitian matrix. 
Further, Hermitian matrices can be decomposed into weighted sums of Pauli terms, i.e., $M\left(\omega\right) = \sum_pm_p\left(\omega\right)h_p$ with $m_p\left(\omega\right)\in\mathbb{R}$ and $h_p=\bigotimes\limits_{j=0}^{n-1}\sigma_{j, p}$ for $\sigma_{j, p}\in\left\{I, X, Y, Z\right\}$ acting on the $j^{\text{th}}$ qubit. Thus, the gradients of 
$U_k\left(\omega_k\right)$ are given by
\begin{equation*}
\frac{\partial U_k\left(\omega_k\right)}{\partial\omega_k} = \sum\limits_pi \frac{\partial m_{k,p}\left(\omega_k\right)}{\partial\omega_k}U_k\left(\omega_k\right)h_{k_p}.
\end{equation*}

Combining this observation with a circuit structure presented in [Simulating physical phenomena by quantum networks](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.65.042323) allows us to compute the gradient with the evaluation of a single quantum circuit.

In [7]:
# Define the values to be assigned to the parameters
value_dict = {a: np.pi / 4, b: np.pi}
# Define the state as operator flow class type
prob_grad_op = CircuitStateFn(primitive=qc, coeff=1.)
prob_grad = Gradient(method='lin_comb').convert(operator=prob_grad_op, params=params)
# Assign the parameters and evaluate the gradient
prob_grad_result = prob_grad.assign_parameters(value_dict).eval()
print('Probability gradient computed with linear combination of unitaries', prob_grad_result)

Probability gradient computed with linear combination of unitaries [[(1.6653345369377348e-16+0j), (-1.6653345369377348e-16+0j)], [(-0.35355339059327373+0j), (0.3535533905932737+0j)]]


#### Natural Gradient

A special type of first order gradient is the natural gradient which has proven itself useful in classical machine learning and is already being studied in the quantum context. This quantity represents a gradient that is 'rescaled' with the inverse Quantum Fisher Information matrix
$$ QFI ^{-1} \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}.$$

Instead of inverting the QFI, one can also use a least-square solver with or without regularization to solve

$$ QFI x = \frac{\partial\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta}.$$

The implementation supports ridge and lasso regularization with automatic search for a good parameter using [L-curve corner search](https://arxiv.org/pdf/1608.04571.pdf) as well as two types of perturbations of the diagonal elements of the QFI.

The natural gradient can be used instead of the standard gradient with any gradient-based optimizer and/or ODE solver.

In [8]:
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
# A regularization method can be chosen, e.g. ridge or lasso with automatic 
# parameter search
nat_grad = NaturalGradient(grad_method='lin_comb', regularization='ridge').convert(
    operator=op, params=params)
# Assign the parameters and evaluate the gradient
nat_grad_result = nat_grad.assign_parameters(value_dict).eval()
print('Natural gradient computed with linear combination of unitaries', nat_grad_result)

Natural gradient computed with linear combination of unitaries [-2.17418416  1.90241114]


### Second Order Gradients

Four types of second order gradients are supported by the gradient framework.
1. Gradient of an expectation value w.r.t. a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega^2} $
2.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta^2} $
3.  Gradient of sampling probabilities w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter, i.e.
 $ \frac{\partial^2 p_i}{\partial\theta^2} = \frac{\partial^2\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta^2} $
4.  Gradient of an expectation value w.r.t. a state $|\psi\left(\theta\right)\rangle$ parameter and a coefficient of the measurement operator respectively observable $\hat{O}\left(\omega\right)$, i.e.
 $ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta\partial\omega} $

#### Hessians w.r.t. Measurement Operator Coefficients

Given a parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$, and observable $\hat{O}\left(\omega\right)=\sum_{ij}\omega_i\omega_j\hat{O}_{ij}$, we want to compute 
$$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\omega_i\partial\omega_j} = \langle\psi\left(\theta\right)|\hat{O}_{ij}\left(\omega\right)|\psi\left(\theta\right)\rangle. $$

In [9]:
# Instantiate the Hamiltonian observable
coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = coeff_0*coeff_1 * X

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

# Define the coefficient tuple w.r.t. which we want to compute the gradient
hessian_coeffs = (coeff_0, coeff_1)

# Convert the operator and the hessian target coefficients into the respective operator
hessian = Hessian().convert(operator = op, params = hessian_coeffs)

# Define the values to be assigned to the parameters
value_dict = {coeff_0: 0.5, coeff_1: -1}

# Assign the parameters and evaluate the gradient
hessian_result = hessian.assign_parameters(value_dict).eval()
print('Hessian ', hessian_result)

Hessian  (1+1.22e-16j)


#### Hessians w.r.t. Measurement Quantum State Parameters
The supported Hessians w.r.t. $\theta$ are
 $$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta^2}, $$
 
and
 
  $$ \frac{\partial^2 p_i}{\partial\theta} = \frac{\partial\langle\psi\left(\theta\right)|i\rangle\langle i|\psi\left(\theta\right)\rangle}{\partial\theta^2}. $$

Notably, the latter case does not use $2^n$ projective operators but is based on sampled values from adaptions of $|\psi\left(\theta\right)\rangle$. 

Just as for gradients these Hessians can be computed with:
* Parameter shifts: This method does not require additional working qubits but the evaluation of multiple expectation values.
* Finite difference: This method compute a numerical approximation rather than an analytic gradient. Equivalently, to the parameter shifting this method does not require additional working qubits but the evaluation of multiple expectation values.
* Linear combination of unitaries: This method requires a single circuit to compute the gradient but uses additional working qubits and intercepting controlled quantum gates.

##### Parameter Shift Hessians

We consider the same setting as for [Parameter Shift Gradients](#param_shift_grad).

For second order gradients, we only need to convolute two $\pi/2$-shifts, i.e.,
 $$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta_i\partial\theta_j} = 4 \left(\langle\psi\left(\theta_i+\pi/2,\theta_j+\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i+\pi/2,\theta_j+\pi/2\right)\rangle - 
 \langle\psi\left(\theta_i+\pi/2,\theta_j-\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i+\pi/2,\theta_j-\pi/2\right)\rangle -
 \langle\psi\left(\theta_i-\pi/2,\theta_j+\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i-\pi/2,\theta_j+\pi/2\right)\rangle
 +
 \langle\psi\left(\theta_i-\pi/2,\theta_j-\pi/2\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i-\pi/2,\theta_j-\pi/2\right)\rangle\right).$$
 Probability gradients are computed equivalently.

In [10]:
# Instantiate a Hamiltonian observable with fixed coefficient values
H = 0.5 * X - 1 * Z

# Instantiate the quantum state with two parameters
a = Parameter('a')
b = Parameter('b')

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

# Define the parameters for which we aim to compute the Hessian
params = [(a,a),(a, b),(b,b)]

# Combine the observable and the state to a ComposedOp
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

# Get the operator object representing the Hessian
state_hess = Hessian(method='param_shift').convert(operator=op, params=params)
values_dict = {a: np.pi / 4, b: np.pi/4}

# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(values_dict).eval()
print('Hessian computed using the parameter shift method', hessian_result)

Hessian computed using the parameter shift method [(0.14644660940672616+4.8625e-17j), (-0.49999999999999994-1.0425e-16j), (0.5+5.55e-17j)]


##### Finite Difference Hessians

We consider the same setting as for [Finite Difference Gradients](#fin_diff_grad).
For second order gradients, we only need to convolute two $\epsilon$-shifts, i.e.,
 $$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta_i\partial\theta_j} = \frac{1}{4\epsilon^2} \left(\langle\psi\left(\theta_i+\epsilon,\theta_j+\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i+\epsilon,\theta_j+\epsilon\right)\rangle - 
 \langle\psi\left(\theta_i+\epsilon,\theta_j-\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i+\epsilon,\theta_j-\epsilon\right)\rangle -
 \langle\psi\left(\theta_i-\epsilon,\theta_j+\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i-\epsilon,\theta_j+\epsilon\right)\rangle
 +
 \langle\psi\left(\theta_i-\epsilon,\theta_j-\epsilon\right)|\hat{O}\left(\omega\right)|\psi\left(\theta_i-\epsilon,\theta_j-\epsilon\right)\rangle\right).$$


 Probability gradients are computed equivalently.

In [11]:
# Get the operator object representing the Hessian using finite difference
state_hess = Hessian(method='fin_diff').convert(operator=op, params=params)
values_dict = {a: np.pi / 4, b: np.pi/4}

# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(values_dict).eval()
print('Hessian computed with finite difference', hessian_result)

Hessian computed with finite difference [(0.1463470458984375+4.1750000000000005e-05j), (-0.4999542236328125-2.499999999999993e-07j), (0.499969482421875+2.7875e-05j)]


##### Linear Combination of Unitaries Hessians
We consider the same setting as for [Linear Combination of Unitaries Gradients](#lin_comb_grad).
Now, second order gradients require two additional working qubits as described in the following [paper](https://arxiv.org/abs/1804.08641).

In [12]:
# Get the operator object representing the Hessian
prob_hess = Hessian(method='lin_comb').convert(operator=op, params=params)
values_dict = {a: np.pi / 4, b: np.pi/4}

# Assign the parameters and evaluate the Hessian
hessian_result = prob_hess.assign_parameters(values_dict).eval()
print('Hessian computed using the linear combination of unitaries method', hessian_result)

Hessian computed using the linear combination of unitaries method [(0.1464466094067261+1.7e-16j), (-0.5-4.76e-16j), (0.5+2.92e-16j)]


#### Hessians w.r.t. Measurement Operator Coefficients and Quantum State Parameters

To compute a Hessian with one parameter from the measurement operator $\hat{O}\left(\omega\right)=\sum_{i}\omega_i\hat{O}_i$ and one parameter from the quantum state $|\psi\left(\theta\right)\rangle$ a gradient w.r.t. a measurement operator coefficient is combined with gradient w.r.t. a state parameter, i.e.,
 $$ \frac{\partial^2\langle\psi\left(\theta\right)|\hat{O}\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta\partial\omega} = \frac{\partial\langle\psi\left(\theta\right)|\hat{O}_i\left(\omega\right)|\psi\left(\theta\right)\rangle}{\partial\theta},$$
 whereby the gradient w.r.t. the state parameter may be computed with either the [parameter shifting](#param_shift_grad), [finite difference](#fin_diff_grad), or the [linear combination of unitaries](#lin_comb_grad).

In [13]:
# Instantiate a Hamiltonian observable with a variable coefficient
coeff_0 = Parameter('c_0')
H = coeff_0 * X

# Instantiate the quantum state with one parameters
a = Parameter('a')

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])

# Define the parameters for which we aim to compute the Hessian
params = (coeff_0, a)

# Combine the observable and the state to a ComposedOp
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

# Get the operator object representing the Hessian
state_hess = Hessian(method='param_shift').convert(operator=op, params=params)
values_dict = {a: np.pi / 4, coeff_0: np.pi/2}

# Assign the parameters and evaluate the Hessian
hessian_result = state_hess.assign_parameters(values_dict).eval()
print('Hessian computed using the parameter shift method', hessian_result)

Hessian computed using the parameter shift method (-0.7071067811865475-1.945e-16j)


### QFI
The Quantum Fisher Information is a metric tensor which is representative for the representation capacity of a 
parameterized quantum state $|\psi\left(\theta\right)\rangle = V\left(\theta\right)|\psi\rangle$ with input state $|\psi\rangle$, parametrized Ansatz $V\left(\theta\right)$.

The entries of the QFI for a pure state reads

$$
QFI_{kl} = 4 * \text{Re}\left[\langle\partial_k\psi|\partial_l|psi\rangle-\langle\partial_k\psi|\psi\rangle\langle\psi|\partial_l\psi\rangle \right].$$

#### Full QFI
To compute the full QFI, we use a working qubit as well as intercepting controlled gates. See e.g. [Variational ansatz-based quantum simulation of imaginary time evolution ](https://www.nature.com/articles/s41534-019-0187-2).

In [14]:
# Instantiate the quantum state with two parameters
a = Parameter('a')    
b = Parameter('b')

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

# Define the parameters for which we want to get the QFI
params = [a, b]

# Wrap the quantum circuit into a CircuitStateFn
state = CircuitStateFn(primitive=qc, coeff=1.)

# Convert the state and the parameters into the operator object that represents the QFI 
qfi = QFI().convert(operator=state, params=params)
# Define the values for which the QFI is to be computed
values_dict = {params[0]: np.pi / 4, params[1]: 0.1}

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('full  QFI ', qfi_result)

full  QFI  [[(0.9999999999999999+9.18e-16j), (-1.9798989873223362e-17+0j)], [(-1.9798989873223362e-17+0j), (0.49999999999999967+4.59e-16j)]]


#### Block-diagonal and Diagonal Approximation
A block-diagonal resp. diagonal approximation of the QFI can be computed without additional working qubits.
This implementation requires the unrolling into Pauli rotations and unparameterized Gates.

In [15]:
# Convert the state and the parameters into the operator object that represents the QFI 
# and set the approximation to 'block_diagonal'
qfi = QFI('block_diag').convert(operator=state, params=params)

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('block-diagonal QFI ', qfi_result)

# Convert the state and the parameters into the operator object that represents the QFI 
# and set the approximation to 'diagonal'
qfi = QFI('diag').convert(operator=state, params=params)

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
print('diagonal QFI ', qfi_result)

block-diagonal QFI  [[1.  0. ]
 [0.  0.5]]
diagonal QFI  [[1.  0. ]
 [0.  0.5]]


### Application Examples

#### VQE with first and second order gradient based optimization

Note: The operator flow's eval method is used to evaluate the function, gradient and Hessian values but we could also use any Backend/QuantumInstance.

In [16]:
from qiskit.aqua.operators import I, X, Z
from qiskit.circuit import QuantumCircuit, ParameterVector
from scipy.optimize import minimize

h2_hamiltonian = -1.05 * (I ^ I) + 0.39 * (I ^ Z) - 0.39 * (Z ^ I) - 0.01 * (Z ^ Z) + 0.18 * (X ^ X)
h2_energy = -1.85727503

# Define the Ansatz
wavefunction = QuantumCircuit(2)
params = ParameterVector('theta', length=8)
it = iter(params)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)
wavefunction.cx(0, 1)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)

op = ~StateFn(h2_hamiltonian) @ StateFn(wavefunction)

In [17]:
def fun(param_values):
    param_dict = dict(zip(params, param_values)) 
    return op.assign_parameters(param_dict).eval()


jac = Gradient(method = 'param_shift').gradient_wrapper(op, bind_params = params)
# param_tuples = [[(param0, param1) for param0 in params] for param1 in params]
hess = Hessian(method = 'param_shift').gradient_wrapper(op, bind_params = params)

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, hess=hess, options={'maxiter': 10, 'xtol': 1e-8})

print('VQE using the gradient and the Hessian:', result['fun'], 'Reference:', h2_energy)

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, options={'maxiter': 10, 'xtol': 1e-8})

print('VQE using the gradient:', result['fun'], 'Reference:', h2_energy)

# result = minimize(fun, np.random.rand(len(params)), method='CG', options={'maxiter': 10, 'tol': 1e-8})

# print('gradient-free VQE:', result['fun'], 'Reference:', h2_energy)

/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/linesearch.py:169: ComplexWarning: Casting complex values to real discards the imaginary part
  amin, amax, isave, dsave)
/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/scipy/optimize/_differentiable_functions.py:158: ComplexWarning: Casting complex values to real discards the imaginary part
  self.x = np.atleast_1d(x).astype(float)


VQE using the gradient and the Hessian: -1.8404995618042734 Reference: -1.85727503
VQE using the gradient: -1.840499843846824 Reference: -1.85727503


In [18]:
from qiskit.aqua.operators import CircuitSampler
from qiskit import Aer
def fun(param_values):
    param_dict = dict(zip(params, param_values))
    param_dict = {k: [v] for k, v in param_dict.items()}
    sampler = CircuitSampler(backend=Aer.get_backend('qasm_simulator')).convert(op, params=param_dict)
    return sampler.eval()


jac = Gradient(method = 'param_shift').gradient_wrapper(op, bind_params = params, backend=Aer.get_backend('qasm_simulator'))
hess = Hessian(method = 'param_shift').gradient_wrapper(op, bind_params = params, backend=Aer.get_backend('qasm_simulator'))

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, hess=hess, 
                  options={'maxiter': 10, 'xtol': 1e-8})

print('VQE using the gradient and the Hessian:', result['fun'])

result = minimize(fun, np.random.rand(len(params)), method='Newton-CG', jac=jac, options={'maxiter': 10, 'xtol': 1e-8})

print('VQE using the gradient:', result['fun'])

TypeError: unsupported operand type(s) for *: 'complex' and 'ParameterExpression'

#### Gibbs state preparation using Variational Quantum Imaginary Time Evolution (VarQITE)

In [19]:
from qiskit.circuit.library import RealAmplitudes

# Temperature
T = 5

# Evolution time
t =  1/(2*T)

# Define the model Hamiltonian
H = SummedOp([0.3 * Z^Z^ I^I, 0.2 * Z^I^ I^I, - 0.5 * I ^ Z^ I^I])

# Instantiate the model ansatz
depth = 1
entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(4, reps=depth, entanglement = entangler_map)
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(2* H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.

In [20]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz    
op = ~StateFn(H) @ CircuitStateFn(ansatz)

# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)

# Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
nat_grad = NaturalGradient(grad_method = 'lin_comb', regularization = 'ridge').convert(op, ansatz.ordered_parameters)

param_values = param_values_init
# Propagate the Ansatz parameters step by step according to the explicit Euler method
for step in time_steps:
    param_dict = dict(zip(ansatz.ordered_parameters, param_values))
    nat_grad_result = np.real(nat_grad.assign_parameters(param_dict).eval())
    param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))

param_dict_final = dict(zip(ansatz.ordered_parameters, param_values))

In [21]:
print(CircuitStateFn(ansatz).assign_parameters(param_dict_final).eval().primitive.data)  

[ 4.92868498e-01-2.61001663e-05j -1.62370068e-04-1.44421499e-05j
  4.15139078e-03+7.81872493e-05j  3.16061558e-03+1.22911048e-04j
 -2.43681368e-04-1.39611042e-05j  4.52932096e-01+6.24231089e-03j
  4.16299040e-03+1.07035747e-04j  3.15210261e-03+1.00883243e-04j
  3.41253168e-03+1.13545538e-05j  3.14409875e-03+7.58834253e-05j
  5.91397045e-01+9.08177619e-03j  5.94607883e-03+1.99581333e-04j
  3.42133743e-03+3.52270766e-05j  3.13629899e-03+5.38329907e-05j
  7.89773889e-03+1.61416301e-04j  4.49112028e-01+1.28116228e-02j]


In [22]:
param_values

[-0.09564177243626364,
 -0.09353485735472458,
 0.013672606137439283,
 0.013891660369470694,
 0.002906980252854886,
 0.0034908675343405,
 0.0034579839251094634,
 0.003458019638009921,
 1.47920223770568,
 1.6661896560514293,
 0.014281853263955466,
 0.01405207464086822,
 0.01375221342713279,
 0.015101461806087934,
 0.0034585748930048175,
 0.003457901839173946]